# LAB07: Answering Business Question

In [ ]:
%pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Answering Business Question').getOrCreate()

### Read youtube dataset

In [ ]:
#https://www.dropbox.com/s/wk8v3ly5mclsj5d/stg_youtube_1K.csv
#https://www.dropbox.com/s/i2rjdae18k8fd0j/stg_youtube.csv 

In [ ]:
!mkdir -p dataset

In [ ]:
!wget -P dataset https://www.dropbox.com/s/i2rjdae18k8fd0j/stg_youtube.csv 

In [ ]:
! ls -lah dataset

In [ ]:
! head dataset/stg_youtube.csv

In [ ]:
from pyspark.sql.types import *

file_path = "dataset/stg_youtube.csv"
youtube_schema=StructType([
    StructField("video_id", StringType(), True),
    StructField("trending_date", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("channel_title", StringType(), True),
    StructField("category_id", IntegerType(), True),
    StructField("publish_time", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("views", IntegerType(), True),
    StructField("likes", IntegerType(), True),
    StructField("dislikes", IntegerType(), True),
    StructField("comment_count", IntegerType(), True)
])

df_stg_youtube = spark.read \
 .option("delimiter", "\t") \
 .schema(youtube_schema) \
 .option("inferSchema", "True") \
 .csv(file_path)

In [ ]:
print ("Total Records: " + str( df_stg_youtube.count()))
df_stg_youtube.show(3, truncate=False)

### Read video_category

In [ ]:
!wget -P dataset https://www.dropbox.com/s/b7wut6mj9wpfyi1/video_category.dat

In [ ]:
! ls -lah dataset

In [ ]:
! head dataset/video_category.dat

In [ ]:
! wc -l dataset/video_category.dat

In [ ]:
file_path = "dataset/video_category.dat"
youtube_vd_cat_schema=StructType([
    StructField("category_id", IntegerType(), True),
    StructField("category_name", StringType(), True),
])


youtube_video_cat = spark.read \
 .option("delimiter", "|") \
 .schema(youtube_vd_cat_schema) \
 .option("inferSchema", "True") \
 .csv(file_path)

print ("Total Records: " + str( youtube_video_cat.count()))
youtube_video_cat.show(5, truncate=True)

### JOIN stg_youtube and video_category

In [ ]:
df_stg_youtube.printSchema()

In [ ]:
youtube = df_stg_youtube.join(youtube_video_cat, on=['category_id'], how='inner')

In [ ]:
df_stg_youtube.count()

In [ ]:
youtube.count()

In [ ]:
youtube.show(5)

In [ ]:
print ("Total Records: " + str( youtube.count()))
print("Number of Columns: " + str(len(youtube.columns)))

In [ ]:
youtube.select("trending_date", "video_id", "channel_title").show(5)

In [ ]:
youtube.createOrReplaceTempView("youtube")

### AGGREGATE

In [ ]:
spark.sql("\
select video_id, max(title), max(category_name), count(*) num_days, max(views) views, max(likes) likes, max(dislikes) dislikes, max(comment_count) num_comment from youtube group by video_id limit 5\
").show()

In [ ]:
video = spark.sql("\
select video_id, max(title) title, max(category_name) category, count(*) num_days, max(views) views, max(likes) likes, max(dislikes) dislikes, max(comment_count) num_comment from youtube group by video_id \
")

In [ ]:
video.createOrReplaceTempView("video")

### Top 10 Most Viewed Video

In [ ]:
spark.sql("\
...
").show(truncate = False)

### Num max days trending

In [ ]:
spark.sql("\
...
").show(truncate = False)

### Top 10 most commented video

In [ ]:
spark.sql("\
select title, category, num_comment from video order by num_comment desc limit 10 \
").show(truncate = False)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

video \
  .orderBy('num_comment', ascending=False) \
  .select(col("title"), col("category"), col("num_comment")) \
  .show(10, truncate = False)

In [ ]:
video \
  .orderBy('num_comment', ascending=False) \
  .select(col("title"), col("category"), col("num_comment")) \
  .toPandas().head(10)

### Top 5 most viewed video

In [ ]:
spark.sql("\
select category, sum(views) total_view, avg(views) avg_view, count(1) num_video from video \
group by category \
order by total_view desc limit 5 \
").show(truncate = False)

In [ ]:
video \
  .groupBy("category") \
  .agg( \
      F.sum(col('views')).alias('total_views'), \
      F.avg(col('views')).alias('avg_views'), \
      F.count(F.lit(1)).alias('num_video') \
      ) \
  .orderBy('total_views', ascending=False).show()

### Data Visualization in Seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

result_pd_df = spark.sql(" \
select category, count(*) num_video from video \
group by category \
order by num_video desc \
").toPandas()

sns.set_style("whitegrid")
ax = sns.barplot(y='category',x='num_video', \
                 data=result_pd_df, \
                 orient='h' \
                )
plt.xlabel("Number of Videos")
plt.ylabel("Categories")
plt.title("Catogories of trend videos")

In [ ]:
df2 = spark.sql("SELECT category, sum(likes) as total_likes, \
                             sum(dislikes) as total_dislikes, \
                             (sum(likes)/sum(dislikes)) as ratio \
                      FROM video \
                      GROUP BY category \
                      ORDER BY ratio DESC" \
                    )
                    
result_pd_df = df2.toPandas()
result_pd_df.head(20)

In [ ]:
sns.set_style("whitegrid")
ax = sns.barplot(y='category',x='ratio', \
                 data=result_pd_df, \
                 orient='h' \
                )
plt.xlabel("Ratio")
plt.ylabel("Categories")
plt.title("Ratio like vs dislike video")